## Data Preprocessing

In this notebook, we will be examining the dataset and handling typical issues such as missing data, feature transformations and potential outliers.

### Importing Libraries

In [29]:
# Standard Packages
from scipy import stats
import pandas as pd
import numpy as np
import pickle

# User Defined Functions
from mlp import visualisation as viz 

# Basic Visualisation Tools
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.offsetbox import AnchoredText


# Interactive Visualisation Tools
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Notebook Settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Data Cleaning and Feature Engineering

In [3]:
df = pd.read_csv('./data/scooter_rental_data.csv')

In [2]:
# Checking for Missing Data
def missing_pct(df, assign=False):
    '''
    Function that shows variables that have missing values and the percentage of total observations that are missing.
    Pass assign=True if user wishes to save the resulting series.
    '''
    
    percentage_missing = df.isnull().mean().sort_values(ascending=False) * 100
    percentage_missing = percentage_missing.loc[percentage_missing > 0]
    
    if len(percentage_missing) > 0:
        print('The following variables have missing data and the percentage of total missing are as follows:')
        print(percentage_missing)
    
    else:
        print('The dataframe has no missing values in any column.')
    
    if assign:
        return percentage_missing

In [4]:
missing_pct(df)

The dataframe has no missing values in any column.


In [5]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace("-", "_")

In [6]:
# Creating target variable
df['total_users'] = df.guest_users + df.registered_users

In [7]:
# Creating log transformed target variable
df['log_total_users'] = np.log(df.total_users)

In [8]:
# Fixing the typos for the weather variable
df.weather = df.weather.str.lower()

df.weather = df.weather.apply(lambda x:
                              'clear' if x == 'lear'
                              else 'cloudy' if x == 'loudy'
                              else 'light snow/rain' if x == 'heavy snow/rain'
                              else x)

In [9]:
# Changing date to datetime format
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')

In [10]:
# Creating additional time related features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['dayofweek'] = df['date'].dt.day_name()

In [11]:
# Creating a variable for the season of the year
seasons = ['winter', 'winter', 'spring', 'spring', 'spring', 'summer',
           'summer', 'summer', 'autumn', 'autumn', 'autumn', 'winter']

month_to_season = dict(zip(range(1,13), seasons))

df['season'] = df['date'].dt.month.map(month_to_season)

In [12]:
# Creating a variable for the difference between temp and feels-like temp
df['temp_diff'] = np.abs(df.temperature - df.feels_like_temperature)

In [13]:
# Changing Variables to categorical
categorical_vars = df.select_dtypes(include='object').columns.to_list()
categorical_vars += ['hr', 'year']

for var in categorical_vars:
    df[var] = df[var].astype('category')

In [15]:
df = pd.get_dummies(df, drop_first=True)

In [16]:
df.head()

,date,temperature,feels_like_temperature,relative_humidity,windspeed,psi,guest_users,registered_users,total_users,log_total_users,year,temp_diff,hr_1,hr_2,hr_3,hr_4,hr_5,hr_6,hr_7,hr_8,hr_9,hr_10,hr_11,hr_12,hr_13,hr_14,hr_15,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,weather_cloudy,weather_light snow/rain,month_August,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September,dayofweek_Monday,dayofweek_Saturday,dayofweek_Sunday,dayofweek_Thursday,dayofweek_Tuesday,dayofweek_Wednesday,season_spring,season_summer,season_winter
0,2011-01-01,66.7,95.0,81.0,0.0,3,21,164,185,5.220356,2011,28.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,2011-01-01,65.0,93.2,80.0,0.0,35,54,417,471,6.154858,2011,28.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,2011-01-01,65.0,93.2,80.0,0.0,38,35,356,391,5.968708,2011,28.2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,2011-01-01,66.7,95.0,75.0,0.0,35,19,129,148,4.997212,2011,28.3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,2011-01-01,66.7,95.0,75.0,0.0,11,-2,10,8,2.079442,2011,28.3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [17]:
# Creating lagged terms up to the 12th order for the logged target
df['l1_log_total_users'] = df.log_total_users.shift(1)
df['l2_log_total_users'] = df.log_total_users.shift(2)
df['l3_log_total_users'] = df.log_total_users.shift(3)
df['l4_log_total_users'] = df.log_total_users.shift(4)
df['l5_log_total_users'] = df.log_total_users.shift(5)
df['l6_log_total_users'] = df.log_total_users.shift(6)
df['l7_log_total_users'] = df.log_total_users.shift(7)
df['l8_log_total_users'] = df.log_total_users.shift(8)
df['l9_log_total_users'] = df.log_total_users.shift(9)
df['l10_log_total_users'] = df.log_total_users.shift(10)
df['l11_log_total_users'] = df.log_total_users.shift(11)
df['l12_log_total_users'] = df.log_total_users.shift(12)

In [18]:
# Droping rows with NaN values due to lagged term creation 
df = df.dropna().reset_index(drop=True)

In [19]:
df = df.drop(['date', 'feels_like_temperature'], axis=1)

In [21]:
df.head()

,temperature,relative_humidity,windspeed,psi,guest_users,registered_users,total_users,log_total_users,year,temp_diff,hr_1,hr_2,hr_3,hr_4,hr_5,hr_6,hr_7,hr_8,hr_9,hr_10,hr_11,hr_12,hr_13,hr_14,hr_15,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,weather_cloudy,weather_light snow/rain,month_August,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September,dayofweek_Monday,dayofweek_Saturday,dayofweek_Sunday,dayofweek_Thursday,dayofweek_Tuesday,dayofweek_Wednesday,season_spring,season_summer,season_winter,l1_log_total_users,l2_log_total_users,l3_log_total_users,l4_log_total_users,l5_log_total_users,l6_log_total_users,l7_log_total_users,l8_log_total_users,l9_log_total_users,l10_log_total_users,l11_log_total_users,l12_log_total_users
0,81.9,77.0,19.0,33,204,711,915,6.818924,2011,29.3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,6.352629,5.986452,4.867534,4.644391,3.465736,2.833213,2.484907,2.079442,4.997212,5.968708,6.154858,5.220356
1,85.3,72.0,20.0,2,328,608,936,6.841615,2011,29.5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,6.818924,6.352629,5.986452,4.867534,4.644391,3.465736,2.833213,2.484907,2.079442,4.997212,5.968708,6.154858
2,85.3,72.0,19.0,12,247,928,1175,7.069023,2011,29.5,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,6.841615,6.818924,6.352629,5.986452,4.867534,4.644391,3.465736,2.833213,2.484907,2.079442,4.997212,5.968708
3,83.6,77.0,20.0,41,279,908,1187,7.079184,2011,29.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,7.069023,6.841615,6.818924,6.352629,5.986452,4.867534,4.644391,3.465736,2.833213,2.484907,2.079442,4.997212
4,81.9,82.0,20.0,19,289,680,969,6.876265,2011,29.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,7.079184,7.069023,6.841615,6.818924,6.352629,5.986452,4.867534,4.644391,3.465736,2.833213,2.484907,2.079442
